<a href="https://colab.research.google.com/github/haiy/all-notebooks/blob/main/Chatbot_Arena_MLE_Elo_Rating_(Bradley_Terry_model)_Calculation_(Jan_26%2C_2024).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this notebook, we present data analysis on Chatbot Arena data collected from https://arena.lmsys.org between April 24, 2023 to Jan 9, 2024.

We explain different Elo calculation methods (online Elo and MLE Elo, also known as Bradley-Terry model) for model ranking.

To view the latest leaderboard, see https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard.


In [ ]:
!pip install gdown==4.6.3

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3


In [ ]:
from collections import defaultdict
import json, math, gdown
import numpy as np
import pandas as pd
import plotly.express as px
from tqdm import tqdm
pd.options.display.float_format = '{:.2f}'.format

# Obtaining and Cleaning the Tournament Data
We are hosting the initial tournament results as a JSON file on Google Drive. We use the `gdown` function to download the data. The data contains all the battels and voting results collected for ranking models.

In [ ]:
url = "https://drive.google.com/file/d/1O3GGotY8I5d4xgxU-qQyizgKFjpSt4ta/view"
filename = gdown.download(url, quiet=False, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1O3GGotY8I5d4xgxU-qQyizgKFjpSt4ta
To: /content/clean_battle_20240125.json
100%|██████████| 68.2M/68.2M [00:00<00:00, 93.3MB/s]


In [ ]:
battles = pd.read_json(filename).sort_values(ascending=True, by=["tstamp"])
battles

,question_id,model_a,model_b,winner,judge,turn,anony,language,tstamp
0,58210e39b3fd4441a2bd4a518bb44c2d,chatglm-6b,koala-13b,model_b,arena_user_1011,1,True,English,1682351591.13
1,2564acd09e3942fd97657d05282d4389,oasst-pythia-12b,alpaca-13b,tie,arena_user_1011,1,True,English,1682351654.67
2,90bfd142157948aba01931726c888e7f,koala-13b,oasst-pythia-12b,model_b,arena_user_1011,1,True,English,1682351708.94
3,a7c5accc53e649a3bc6b2e41d962ebc4,vicuna-13b,oasst-pythia-12b,model_b,arena_user_1011,1,True,English,1682351785.19
4,adf27e819a3c494cb6e993f0c660e097,vicuna-13b,koala-13b,model_a,arena_user_1011,1,True,English,1682351891.66
...,...,...,...,...,...,...,...,...,...
244973,24f5e23d67a54544b184ded2f52ea982,gpt-3.5-turbo-0613,stripedhyena-nous-7b,model_a,arena_user_104418,1,True,German,1706198238.27
244974,044e0ae3228649849b60e94d7ef556de,gpt-4-turbo,gpt-4-0613,model_b,arena_user_101378,1,True,Russian,1706198264.06
244975,bc78f481c13e45669058a9c1a105a99b,stripedhyena-nous-7b,llama-2-7b-chat,tie,arena_user_104053,1,True,Japanese,1706198280.29
244976,122016157cc54d49ad5575dc8cf0bf0a,deepseek-llm-67b-chat,vicuna-33b,model_a,arena_user_104420,1,True,English,1706198351.40


# Exploratory Analysis

Before computing the Elo ratings, we first conduct some basic exploratory analysis to highlight a few key properties and caveates with this data.

## Statistics

We allowed the user to declare a tie between the pairs of models.  To collect additional data, later in the tournament we also allowed the user to declare a tie in which both models were bad.  There were a significant portion of tied outcomes.

In [ ]:
fig = px.bar(battles["winner"].value_counts(),
             title="Counts of Battle Outcomes", text_auto=True, height=400)
fig.update_layout(xaxis_title="Battle Outcome", yaxis_title="Count",
                  showlegend=False)
fig

In [ ]:
battles_no_ties = battles[~battles["winner"].str.contains("tie")]

## Non-uniform Model Frequency

The model frequency is not uniform because of the follwoing reasons:
- Several different matching and sampling algorithms were used. We employed uniform sampling as well as weighted sampling methods, which assign greater weights to better models.
- Some new models were added later.


In [ ]:
fig = px.bar(pd.concat([battles["model_a"], battles["model_b"]]).value_counts(),
             title="Battle Count for Each Model", text_auto=True)
fig.update_layout(xaxis_title="model", yaxis_title="Battle Count", height=400,
                  showlegend=False)
fig

We examing the number of pairings for each combination of models.

In [ ]:
def visualize_battle_count(battles, title, show_num_models=30):
    ptbl = pd.pivot_table(battles, index="model_a", columns="model_b", aggfunc="size",
                          fill_value=0)
    battle_counts = ptbl + ptbl.T
    ordering = battle_counts.sum().sort_values(ascending=False).index
    ordering = ordering[:show_num_models]
    fig = px.imshow(battle_counts.loc[ordering, ordering],
                    title=title, text_auto=True)
    fig.update_layout(xaxis_title="Model B",
                      yaxis_title="Model A",
                      xaxis_side="top", height=800, width=800,
                      title_y=0.07, title_x=0.5,
                      font=dict(size=10))
    fig.update_traces(hovertemplate=
                      "Model A: %{y}<br>Model B: %{x}<br>Count: %{z}<extra></extra>")
    return fig

fig = visualize_battle_count(battles, title="Battle Count of Each Combination of Models", show_num_models=30)
fig

### Battles Excluding Ties

In [ ]:
visualize_battle_count(battles_no_ties, "Battle Count for Each Combination of Models (without Ties)")

### Counting Ties

In [ ]:
visualize_battle_count(battles[battles['winner'].str.contains("tie")], "Tie Count for Each Combination of Models")

## Inferred Language

We also inferred the language for each conversation using `polyglot` package. This is just an estimate but will help guide future analysis.  The vast majority of conversations were in English.

In [ ]:
topk = 15
fig = px.bar(battles["language"].value_counts().head(topk),
             title=f"Battle Counts for the Top {topk} Languages",
             text_auto=True, height=400)
fig.update_layout(xaxis_title="Language", yaxis_title="Count", showlegend=False)
fig

## Number of Conversation Turns

We also noticed that most counversations only have one turn.

In [ ]:
fig = px.histogram(battles["turn"],
             title=f"Number of Conversation Turns",
             text_auto=True, height=400)
fig.update_layout(xaxis_title="Turns", yaxis_title="Count", showlegend=False)
fig

## Pairwise Win Fractions

Finally, we can also compute the pairwise win fractions. However, because each model can play as Model A and as Model B and win in both situations we need to compute the wins in both configurations divided by the number of pairings of each model.

In [ ]:
def compute_pairwise_win_fraction(battles, max_num_models=30):
    # Times each model wins as Model A
    a_win_ptbl = pd.pivot_table(
        battles[battles['winner'] == "model_a"],
        index="model_a", columns="model_b", aggfunc="size", fill_value=0)

    # Table counting times each model wins as Model B
    b_win_ptbl = pd.pivot_table(
        battles[battles['winner'] == "model_b"],
        index="model_a", columns="model_b", aggfunc="size", fill_value=0)

    # Table counting number of A-B pairs
    num_battles_ptbl = pd.pivot_table(battles,
        index="model_a", columns="model_b", aggfunc="size", fill_value=0)

    # Computing the proportion of wins for each model as A and as B
    # against all other models
    row_beats_col_freq = (
        (a_win_ptbl + b_win_ptbl.T) /
        (num_battles_ptbl + num_battles_ptbl.T)
    )

    # Arrange ordering according to proprition of wins
    prop_wins = row_beats_col_freq.mean(axis=1).sort_values(ascending=False)
    prop_wins = prop_wins[:max_num_models]
    model_names = list(prop_wins.keys())
    row_beats_col = row_beats_col_freq.loc[model_names, model_names]
    return row_beats_col

def visualize_pairwise_win_fraction(battles, title, max_num_models=30):
    row_beats_col = compute_pairwise_win_fraction(battles, max_num_models)
    fig = px.imshow(row_beats_col, color_continuous_scale='RdBu',
                    text_auto=".2f", title=title)
    fig.update_layout(xaxis_title=" Model B: Loser",
                  yaxis_title="Model A: Winner",
                  xaxis_side="top", height=900, width=900,
                  title_y=0.07, title_x=0.5)
    fig.update_traces(hovertemplate=
                  "Model A: %{y}<br>Model B: %{x}<br>Fraction of A Wins: %{z}<extra></extra>")

    return fig

In [ ]:
fig = visualize_pairwise_win_fraction(battles_no_ties,
      title = "Fraction of Model A Wins for All Non-tied A vs. B Battles")
fig

## Preliminary Ranking

Using just the average win rate against all other models we can already compute an estimated leaderboard.
However, this method may not be as scalable as the Elo rating system that we will use later because this method requires data from all model combinations.

In [ ]:
row_beats_col_freq = compute_pairwise_win_fraction(battles_no_ties)
fig = px.bar(row_beats_col_freq.mean(axis=1).sort_values(ascending=False),
             title="Average Win Rate Against All Other Models (Assuming Uniform Sampling and No Ties)",
             text_auto=".2f")
fig.update_layout(yaxis_title="Average Win Rate", xaxis_title="Model",
                  showlegend=False)
fig

#Elo Ratings

The [Elo rating system ](https://en.wikipedia.org/wiki/Elo_rating_system)is a method for calculating the relative skill levels of players, which has been widely adopted in chess and other competitive games. The difference in the ratings between two players serves as a predictor of the outcome of a match. The Elo rating system works well for our case because we have multiple models and we run pairwise battles between them.
In this section, we present different methods for calculating Elo ratings.

### Compute Ratings
We first use the online linear update algorithm to compute Elo ratings.
We choose a small K-factor of 4 to make the Elo ratings more stable and less biased towards recent games.

In [ ]:
def compute_online_elo(battles, K=4, SCALE=400, BASE=10, INIT_RATING=1000):
    rating = defaultdict(lambda: INIT_RATING)

    for rd, model_a, model_b, winner in battles[['model_a', 'model_b', 'winner']].itertuples():
        ra = rating[model_a]
        rb = rating[model_b]
        ea = 1 / (1 + BASE ** ((rb - ra) / SCALE))
        eb = 1 / (1 + BASE ** ((ra - rb) / SCALE))
        if winner == "model_a":
            sa = 1
        elif winner == "model_b":
            sa = 0
        elif winner == "tie" or winner == "tie (bothbad)":
            sa = 0.5
        else:
            raise Exception(f"unexpected vote {winner}")
        rating[model_a] += K * (sa - ea)
        rating[model_b] += K * (1 - sa - eb)

    # calibrate llama-13b to 800
    delta = (800-rating["llama-13b"])
    for model in battles["model_a"].unique():
        rating[model] += delta

    return rating

In [ ]:
def preety_print_model_ratings(ratings):
    df = pd.DataFrame([
        [n, ratings[n]] for n in ratings.keys()
    ], columns=["Model", "Elo rating"]).sort_values("Elo rating", ascending=False).reset_index(drop=True)
    df["Elo rating"] = (df["Elo rating"] + 0.5).astype(int)
    df.index = df.index + 1
    return df

online_elo_ratings = compute_online_elo(battles)
preety_print_model_ratings(online_elo_ratings)

,Model,Elo rating
1,gpt-4-turbo,1137
2,bard-jan-24-gemini-pro,1118
3,gpt-4-0314,1054
4,gpt-3.5-turbo-0314,1046
5,claude-1,1041
6,mistral-medium,1039
7,gpt-4-0613,1038
8,wizardlm-70b,1034
9,claude-2.1,1017
10,yi-34b-chat,1017


However, even with a small K-factor, we still found this online update algorithm to be unstable.

To demonstrate it, we recompute Elo rating by using the reversed game order and observe significant difference due to online update of Elo which biases the recent games.

In [ ]:
def preety_print_two_ratings(ratings_1, ratings_2, column_names):
    df = pd.DataFrame([
        [n, ratings_1[n], ratings_2[n]] for n in ratings_1.keys()
    ], columns=["Model", column_names[0], column_names[1]]).sort_values(column_names[0], ascending=False).reset_index(drop=True)
    df[column_names[0]] = (df[column_names[0]] + 0.5).astype(int)
    df[column_names[1]] = (df[column_names[1]] + 0.5).astype(int)
    df.index = df.index + 1
    return df

elo_mle_ratings_reverse = compute_online_elo(battles.iloc[::-1])
preety_print_two_ratings(online_elo_ratings,
                         elo_mle_ratings_reverse,
                         column_names=["Elo rating", "Elo rating with reverse order"])

,Model,Elo rating,Elo rating with reverse order
1,gpt-4-turbo,1137,1176
2,bard-jan-24-gemini-pro,1118,1140
3,gpt-4-0314,1054,1262
4,gpt-3.5-turbo-0314,1046,1143
5,claude-1,1041,1194
6,mistral-medium,1039,1060
7,gpt-4-0613,1038,1191
8,wizardlm-70b,1034,1043
9,claude-2.1,1017,1031
10,yi-34b-chat,1017,1001



### Maximum Likelihood Estimation for Elo Ratings (aka [Bradley-Terry model](https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model))

In the context of LLM evaluation, models can be assumed to be static. In this case, we can directly fit the ratings by maximum likelihood estimation method (aka Bradley-Terry model), which produce significantly stable ratings. Here we provide an implementation with logistic regression.

In [ ]:
def compute_mle_elo(df, SCALE=400, BASE=10, INIT_RATING=1000):
    from sklearn.linear_model import LogisticRegression
    models = pd.concat([df["model_a"], df["model_b"]]).unique()
    models = pd.Series(np.arange(len(models)), index=models)

    # duplicate battles
    df = pd.concat([df, df], ignore_index=True)
    p = len(models.index)
    n = df.shape[0]

    X = np.zeros([n, p])
    X[np.arange(n), models[df["model_a"]]] = +math.log(BASE)
    X[np.arange(n), models[df["model_b"]]] = -math.log(BASE)

    # one A win => two A win
    Y = np.zeros(n)
    Y[df["winner"] == "model_a"] = 1.0

    # one tie => one A win + one B win
    # find tie + tie (both bad) index
    tie_idx = (df["winner"] == "tie") | (df["winner"] == "tie (bothbad)")
    tie_idx[len(tie_idx)//2:] = False
    Y[tie_idx] = 1.0

    lr = LogisticRegression(fit_intercept=False)
    lr.fit(X,Y)

    elo_scores = SCALE * lr.coef_[0] + INIT_RATING

    # calibrate llama-13b to 800
    elo_scores += (800-elo_scores[models["llama-13b"]])
    return pd.Series(elo_scores, index = models.index).sort_values(ascending=False)

In [ ]:
elo_mle_ratings = compute_mle_elo(battles)
preety_print_model_ratings(elo_mle_ratings)

,Model,Elo rating
1,gpt-4-turbo,1250
2,bard-jan-24-gemini-pro,1214
3,gpt-4-0314,1189
4,gpt-4-0613,1161
5,mistral-medium,1150
6,claude-1,1150
7,claude-2.0,1131
8,mixtral-8x7b-instruct-v0.1,1122
9,gemini-pro-dev-api,1122
10,claude-2.1,1119


### Compute Bootstrap Confidence Interavals for MLE Elo Scores

We can further use bootstrap to estimate the confidence intervals as well.


In [ ]:
def get_bootstrap_result(battles, func_compute_elo, num_round):
    rows = []
    for i in tqdm(range(num_round), desc="bootstrap"):
        rows.append(func_compute_elo(battles.sample(frac=1.0, replace=True)))
    df = pd.DataFrame(rows)
    return df[df.median().sort_values(ascending=False).index]


In [ ]:
BOOTSTRAP_ROUNDS = 100

np.random.seed(42)
bootstrap_elo_lu = get_bootstrap_result(battles, compute_mle_elo, BOOTSTRAP_ROUNDS)

bootstrap: 100%|██████████| 100/100 [08:29<00:00,  5.09s/it]


In [ ]:
def visualize_bootstrap_scores(df, title):
    bars = pd.DataFrame(dict(
        lower = df.quantile(.025),
        rating = df.quantile(.5),
        upper = df.quantile(.975))).reset_index(names="model").sort_values("rating", ascending=False)
    bars['error_y'] = bars['upper'] - bars["rating"]
    bars['error_y_minus'] = bars['rating'] - bars["lower"]
    bars['rating_rounded'] = np.round(bars['rating'], 2)
    fig = px.scatter(bars, x="model", y="rating", error_y="error_y",
                     error_y_minus="error_y_minus", text="rating_rounded",
                     title=title)
    fig.update_layout(xaxis_title="Model", yaxis_title="Rating",
                      height=600)
    return fig

fig = visualize_bootstrap_scores(bootstrap_elo_lu, "Bootstrap of MLE Elo Rating Estimates")
fig

We previously apply bootstrapping on the online Elo to obtain stabler ratings.

In [ ]:
np.random.seed(42)
bootstrap_online_elo = get_bootstrap_result(battles, compute_online_elo, BOOTSTRAP_ROUNDS)

bootstrap: 100%|██████████| 100/100 [01:41<00:00,  1.02s/it]


We can see the bootstrapping medians obtained by both methods are similar.

In [ ]:
preety_print_two_ratings(bootstrap_elo_lu.quantile(.5),
                         bootstrap_online_elo.quantile(.5),
                         column_names=["Bootstrap Median of MLE Elo", "Bootstrap Median of Online Elo"])

,Model,Bootstrap Median of MLE Elo,Bootstrap Median of Online Elo
1,gpt-4-turbo,1249,1251
2,bard-jan-24-gemini-pro,1215,1217
3,gpt-4-0314,1190,1190
4,gpt-4-0613,1161,1162
5,mistral-medium,1150,1152
6,claude-1,1150,1151
7,claude-2.0,1132,1128
8,gemini-pro-dev-api,1123,1122
9,mixtral-8x7b-instruct-v0.1,1123,1123
10,claude-2.1,1119,1117


However, online Elo's confidence intervals are significantly larger than the MLE Elo.

In [ ]:
fig = visualize_bootstrap_scores(bootstrap_online_elo, "Bootstrap of Online Elo Rating Estimates")
fig

### Predict Win Rates
Utilizing Elo ratings allows us to predict win probabilities. By comparing the predicted win rates with the actual win rates, we can gain insight into the accuracy and quality of the Elo rating system.






In [ ]:
def predict_win_rate(elo_ratings, SCALE=400, BASE=10, INIT_RATING=1000):
    names = sorted(list(elo_ratings.keys()))
    wins = defaultdict(lambda: defaultdict(lambda: 0))
    for a in names:
        for b in names:
            ea = 1 / (1 + BASE ** ((elo_ratings[b] - elo_ratings[a]) / SCALE))
            wins[a][b] = ea
            wins[b][a] = 1 - ea

    data = {
        a: [wins[a][b] if a != b else np.NAN for b in names]
        for a in names
    }

    df = pd.DataFrame(data, index=names)
    df.index.name = "model_a"
    df.columns.name = "model_b"
    return df.T

In [ ]:
win_rate = predict_win_rate(dict(bootstrap_elo_lu.quantile(0.5)))
ordered_models = win_rate.mean(axis=1).sort_values(ascending=False).index
ordered_models = ordered_models[:30]
fig = px.imshow(win_rate.loc[ordered_models, ordered_models],
                color_continuous_scale='RdBu', text_auto=".2f",
                title="Predicted Win Rate Using Elo Ratings for Model A in an A vs. B Battle")
fig.update_layout(xaxis_title="Model B",
                  yaxis_title="Model A",
                  xaxis_side="top", height=900, width=900,
                  title_y=0.07, title_x=0.5)
fig.update_traces(hovertemplate=
                  "Model A: %{y}<br>Model B: %{x}<br>Win Rate: %{z}<extra></extra>")
fig

### Compute Bootstrap Confidence Intervals Assuming Uniform Sampling

We also study how the ratings will change if we only sample an equal number of battles for each model pair.

In [ ]:
def sample_battle_even(battles, n_per_battle):
    groups = battles.groupby(["model_a", "model_b"], as_index=False)
    resampled = (groups
                 .apply(lambda grp: grp.sample(n_per_battle, replace=True))
                 .reset_index(drop=True))
    return resampled

In [ ]:
num_samples = 50
battles_even = sample_battle_even(battles, num_samples)
pd.pivot_table(battles_even, index="model_a", columns="model_b", aggfunc="size", fill_value=0)

model_b,RWKV-4-Raven-14B,alpaca-13b,bard-jan-24-gemini-pro,chatglm-6b,chatglm2-6b,chatglm3-6b,claude-1,claude-2.0,claude-2.1,claude-instant-1,...,stripedhyena-nous-7b,tulu-2-dpo-70b,vicuna-13b,vicuna-33b,vicuna-7b,wizardlm-13b,wizardlm-70b,yi-34b-chat,zephyr-7b-alpha,zephyr-7b-beta
model_a,,,,,,,,,,,,,,,,,,,,,
RWKV-4-Raven-14B,0,50,0,50,0,0,50,50,0,50,...,0,0,50,50,50,50,0,0,0,0
alpaca-13b,50,0,0,50,0,0,50,50,0,50,...,0,0,50,50,50,50,0,0,0,0
bard-jan-24-gemini-pro,0,0,0,0,0,50,50,50,50,50,...,50,50,50,50,0,0,50,50,0,50
chatglm-6b,50,50,0,0,0,0,50,50,0,50,...,0,0,50,50,50,50,0,0,0,0
chatglm2-6b,0,50,0,0,0,50,50,50,50,50,...,0,50,50,50,50,50,50,50,50,50
chatglm3-6b,0,0,50,0,50,0,50,50,50,50,...,50,50,50,50,50,50,50,50,0,50
claude-1,50,50,50,50,50,50,0,50,50,50,...,50,50,50,50,50,50,50,50,50,50
claude-2.0,50,50,50,50,50,50,50,0,50,50,...,50,50,50,50,50,50,50,50,50,50
claude-2.1,0,0,50,0,50,50,50,50,0,50,...,50,50,50,50,50,50,50,50,0,50


In [ ]:
# Sampling Battles Evenly
def get_bootstrap_even_sample(battles, n_per_battle, func_compute_elo, num_round=BOOTSTRAP_ROUNDS):
    rows = []
    for n in tqdm(range(num_round), desc="sampling battles evenly"):
        resampled = sample_battle_even(battles, n_per_battle)
        rows.append(func_compute_elo(resampled))
    df = pd.DataFrame(rows)
    return df[df.median().sort_values(ascending=False).index]

In [ ]:
print("number of samples per battle pair:", num_samples)
bootstrap_even_lu = get_bootstrap_even_sample(battles, num_samples, compute_mle_elo, num_round=100)

number of samples per battle pair: 50


sampling battles evenly: 100%|██████████| 100/100 [04:59<00:00,  3.00s/it]


In [ ]:
fig = visualize_bootstrap_scores(bootstrap_even_lu, f"Bootstrap of MLE Elo Estimates - Even sample")
fig

# Language-specific Leaderboards
We present two language-specific leaderboards, by isolating the chat data into two subsets based on the language: (1) English-only and (2) Non-English.

## English-only

In [ ]:
english_only_battles = battles[battles["language"] == "English"]
elo_ratings = compute_mle_elo(english_only_battles)
print(elo_ratings)

gpt-4-turbo                  1255.54
bard-jan-24-gemini-pro       1210.36
gpt-4-0314                   1191.82
gpt-4-0613                   1166.12
mistral-medium               1162.66
claude-1                     1150.89
mixtral-8x7b-instruct-v0.1   1136.36
claude-2.0                   1134.41
yi-34b-chat                  1128.75
gemini-pro-dev-api           1126.96
claude-2.1                   1120.26
gemini-pro                   1119.71
wizardlm-70b                 1117.94
gpt-3.5-turbo-0613           1116.64
claude-instant-1             1116.46
tulu-2-dpo-70b               1113.43
vicuna-33b                   1110.06
deepseek-llm-67b-chat        1102.48
starling-lm-7b-alpha         1101.97
gpt-3.5-turbo-0314           1101.18
llama-2-70b-chat             1100.29
openhermes-2.5-mistral-7b    1092.11
pplx-70b-online              1086.70
llama2-70b-steerlm-chat      1086.21
openchat-3.5                 1084.88
dolphin-2.2.1-mistral-7b     1079.49
solar-10.7b-instruct-v1.0    1078.52
g

## Non-English

In [ ]:
non_english_battles = battles[battles["language"] != "English"]
elo_ratings = compute_mle_elo(non_english_battles)
print(elo_ratings)

gpt-4-turbo                  1227.08
bard-jan-24-gemini-pro       1224.71
gpt-4-0314                   1180.05
claude-1                     1144.54
gpt-4-0613                   1140.08
gpt-3.5-turbo-0314           1123.78
claude-2.0                   1117.72
gpt-3.5-turbo-0613           1113.13
claude-2.1                   1112.05
mistral-medium               1106.93
gemini-pro-dev-api           1100.50
gemini-pro                   1091.27
claude-instant-1             1084.26
deepseek-llm-67b-chat        1083.46
mixtral-8x7b-instruct-v0.1   1072.70
tulu-2-dpo-70b               1071.69
wizardlm-70b                 1062.28
yi-34b-chat                  1055.84
gpt-3.5-turbo-1106           1055.61
llama2-70b-steerlm-chat      1047.04
openchat-3.5                 1046.75
starling-lm-7b-alpha         1042.09
vicuna-33b                   1040.57
pplx-70b-online              1030.22
qwen-14b-chat                1028.56
vicuna-13b                   1018.33
wizardlm-13b                 1017.88
o

# Links



Some good resources to learn more about Elo rating systems:
- Elo rating system https://en.wikipedia.org/wiki/Elo_rating_system
- Bradley-Terry model https://en.wikipedia.org/wiki/Bradley%E2%80%93Terry_model
- An introduction video https://www.youtube.com/watch?v=AsYfbmp0To0
- A FiveThirtyEight article https://fivethirtyeight.com/methodology/how-our-nfl-predictions-work/
